In [1]:
import gym
import ray
import numpy as np
import pandas as pd
from tqdm import tqdm
ray.init()

2020-11-03 12:16:37,191	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.100',
 'raylet_ip_address': '192.168.0.100',
 'redis_address': '192.168.0.100:6379',
 'object_store_address': '/tmp/ray/session_2020-11-03_12-16-36_703095_28837/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-03_12-16-36_703095_28837/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-11-03_12-16-36_703095_28837',
 'metrics_export_port': 59821}

Load the current s&p 500 ticker list, we will only be trading on these stocks

In [2]:

tickers = open('s&p500_tickers.dat', 'r').read().split('\n')
print(tickers)

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DL

Data start and end dates and initialise ticker dataframe dictionary

In [3]:
one_day = pd.Timedelta(days=1)

i = 0
cur_day = pd.to_datetime('1992-06-15', format=r'%Y-%m-%d') #pd.to_datetime('1992-06-15')
end_day = pd.to_datetime('2020-01-01', format=r'%Y-%m-%d')

end_df = pd.read_csv('equity_data/' + (end_day - one_day).strftime(r'%Y%m%d') + '.csv')
ticker_df = end_df.loc[end_df.symbol.isin(tickers)] # Tickers that are in the dataframe on the last day
ticker_dict = {ticker_df.symbol.iloc[i] : ticker_df.finnhub_id.iloc[i] for i in range(len(ticker_df.index))} # Create a mapping between tickers and finnhub_ids

For all dates between start and end range, load the day into ticker dict with the key being ticker and dataframe indexed by day

In [4]:

df_columns = pd.read_csv('equity_data/' + cur_day.strftime(r'%Y%m%d') + '.csv').columns
ticker_dfs = { ticker : pd.DataFrame(index=pd.date_range(cur_day, end_day - one_day, freq='D'), columns=df_columns) for ticker in tickers }
save_df = False
if save_df:
    pbar = tqdm(total=(end_day - cur_day).days)
    while (cur_day != end_day):
        pbar.update()
        try:
            day_df = pd.read_csv('equity_data/' + cur_day.strftime(r'%Y%m%d') + '.csv')
        except FileNotFoundError:
            cur_day += one_day
            i += 1
            continue
        for ticker in ticker_dict.keys():
            if ticker in day_df.symbol.values:
                row = day_df.loc[day_df.finnhub_id == ticker_dict[ticker]]
                if row.shape[0] == 2:
                    print(ticker)
                    print(row)
                if not row.shape[0] == 0:
                    ticker_dfs[ticker].loc[cur_day] = row.values[0, :]
        cur_day += one_day
        i += 1
    pbar.close()

In [5]:
# Loading logic, as the above is slow process and we dont want to perform it every time
if save_df:
    for ticker, ticker_frame in ticker_dfs.items():
        ticker_frame.reset_index(inplace=True)
        ticker_frame.to_feather('equity_data/stored/' + ticker.lower() + '.feather')
else:
    print('Loading from storage...')
    for symbol in ticker_dict.keys():
        ticker_dfs[symbol] = pd.read_feather('equity_data/stored/' + symbol.lower() + '.feather').set_index('index', drop=True)
    print(ticker_dfs)

ss    open    high     low   close  \
index                                                                  
2002-10-25  FH109424521   WYNN  None   13.00   13.15   12.85   13.01   
2002-10-28  FH109424521   WYNN  None   13.01   13.05   12.96   13.01   
2002-10-29  FH109424521   WYNN  None   13.00   13.01   12.04   12.64   
2002-10-30  FH109424521   WYNN  None   12.61   12.62   12.13   12.50   
2002-10-31  FH109424521   WYNN  None   12.56   13.02   12.50   12.61   
...                 ...    ...   ...     ...     ...     ...     ...   
2019-12-24  FH109424521   WYNN  None  139.91  140.12  139.24  139.44   
2019-12-26  FH109424521   WYNN  None  140.19  141.00  139.57  140.94   
2019-12-27  FH109424521   WYNN  None  141.17  141.84  138.73  139.81   
2019-12-30  FH109424521   WYNN  None  140.25  140.87  138.02  139.38   
2019-12-31  FH109424521   WYNN  None  138.66  140.07  138.25  138.87   

                volume  div adjustment     bid     ask  
index                                   

In [6]:
# Clear the data somewhat, we only want frames with more than 2000 days that have gaps no larger than 7 days
to_delete = []
for ticker, frame in ticker_dfs.items():
    prev_day = frame.index[-1]
    frame.dropna(axis='index', how='all', inplace=True)
    if frame.empty:
        to_delete.append(ticker)
    elif len(frame.index) < 2000:
        to_delete.append(ticker)
    else:
        for day in frame.index[::-1][1:]:
            if (prev_day - day).days > 7: # if gap between datapoints larger than 7 days, remove
                to_delete.append(ticker)
                break
            prev_day = day

for ticker in to_delete:
    del ticker_dfs[ticker]
    print('Deleting ticker: ' + ticker)
print(len(ticker_dfs.keys()))

Deleting ticker: AAL
Deleting ticker: ABBV
Deleting ticker: ADP
Deleting ticker: AEE
Deleting ticker: ALLE
Deleting ticker: AMCR
Deleting ticker: ANET
Deleting ticker: ANTM
Deleting ticker: APTV
Deleting ticker: BF.B
Deleting ticker: BKNG
Deleting ticker: BKR
Deleting ticker: BRK.B
Deleting ticker: C
Deleting ticker: CARR
Deleting ticker: CBRE
Deleting ticker: CCI
Deleting ticker: CDW
Deleting ticker: CFG
Deleting ticker: CMCSA
Deleting ticker: CMI
Deleting ticker: CNC
Deleting ticker: CNP
Deleting ticker: COP
Deleting ticker: COST
Deleting ticker: COTY
Deleting ticker: CTVA
Deleting ticker: CVS
Deleting ticker: DD
Deleting ticker: DOW
Deleting ticker: DXC
Deleting ticker: EA
Deleting ticker: ES
Deleting ticker: EVRG
Deleting ticker: EXC
Deleting ticker: FANG
Deleting ticker: FB
Deleting ticker: FOX
Deleting ticker: FOXA
Deleting ticker: FTI
Deleting ticker: FTV
Deleting ticker: GL
Deleting ticker: GOOG
Deleting ticker: GOOGL
Deleting ticker: HLT
Deleting ticker: HPE
Deleting ticker: H

In [7]:
# Align dataframes by date and create an intersection
index_intersection = ticker_dfs[list(ticker_dfs.keys())[0]].index
print(index_intersection)
for ticker, ticker_frame in ticker_dfs.items():
    index_intersection = index_intersection.intersection(ticker_frame.index)
    print(ticker + ': ' + str(len(index_intersection)))
print(index_intersection)

DatetimeIndex(['1999-11-18', '1999-11-19', '1999-11-22', '1999-11-23',
               '1999-11-24', '1999-11-26', '1999-11-29', '1999-11-30',
               '1999-12-01', '1999-12-02',
               ...
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', name='index', length=5061, freq=None)
A: 5061
AAP: 4553
AAPL: 4551
ABC: 4551
ABMD: 4550
ABT: 4550
ACN: 4550
ADBE: 4550
ADI: 4550
ADM: 4550
ADSK: 4550
AEP: 4550
AES: 4550
AFL: 4550
AIG: 4550
AIV: 4550
AIZ: 4001
AJG: 4001
AKAM: 4001
ALB: 4001
ALGN: 4001
ALK: 4001
ALL: 4001
ALXN: 4001
AMAT: 4001
AMD: 4001
AME: 4001
AMGN: 4001
AMP: 3585
AMT: 3585
AMZN: 3585
ANSS: 3585
AON: 2537
AOS: 2537
APA: 2537
APD: 2537
APH: 2537
ARE: 2537
ATO: 2537
ATVI: 2537
AVB: 2537
AVGO: 2537
AVY: 2537
AWK: 2537
AXP: 2537
AZO: 2537
BA: 2537
BAC: 2537
BAX: 2537
BBY: 2537
BDX: 2537
BEN: 2537
BIIB

In [8]:
for ticker in ticker_dfs.keys():
    ticker_dfs[ticker] = ticker_dfs[ticker].loc[index_intersection]
print(ticker_dfs)

  
2019-12-30  NaN       None  68.63  68.64  
2019-12-31  NaN       None  69.07  69.09  

[2050 rows x 12 columns], 'WST':              finnhub_id symbol class    open    high     low   close  \
index                                                                  
2011-11-01  FH106641721    WST  None   37.50   38.64   35.86   37.43   
2011-11-02  FH106641721    WST  None   37.74   38.15   37.60   38.05   
2011-11-03  FH106641721    WST  None   38.40   39.28   37.41   39.21   
2011-11-04  FH106641721    WST  None   38.81   39.04   38.45   38.75   
2011-11-07  FH106641721    WST  None   38.64   39.07   37.99   39.04   
...                 ...    ...   ...     ...     ...     ...     ...   
2019-12-24  FH106641721    WST  None  150.99  152.09  150.06  150.73   
2019-12-26  FH106641721    WST  None  151.04  151.12  150.18  150.77   
2019-12-27  FH106641721    WST  None  151.43  152.12  150.40  150.81   
2019-12-30  FH106641721    WST  None  150.87  151.32  149.74  150.81   
2019-12-31  F

In [9]:
# Env config file structure for reference
n_assets = 0
n_features = 0
config = {
    'initial_balance': 0,
    'initial_portfolio': [0]*n_assets,
    'tickers': ['']*n_assets, # Tickers to trade, must correspond to tickers in dataframe dict! Implicitly defines number of assets
    'indicators': [None]*n_features, # Indicator functions/classes to compute features for each stock, implicitly defines number of features. TODO: Support multidimensional indicators
    'max_indicator_lookback': 0, # Number of days after which all indicators can compute proper values
    'trading_days': 0,
    'start_day_offset': None
}

In [10]:
class TradingEnv(gym.Env):

    def __init__(self, env_config):
        super(TradingEnv, self).__init__()

        self._env_config = env_config
        self._tickers = env_config['tickers']
        self._indicator_funcs = self._env_config['indicators']
        self._max_indicator_lookback = self._env_config['max_indicator_lookback'] # Number of days after which all indicators can compute proper values

        self._n_assets = len(self._tickers)
        self._n_features = len(self._indicator_funcs)

        assert self._n_assets != 0, 'Number of assets must not be zero!'
        assert self._n_features != 0, 'Number of features must not be zero!'

        self._df_dict = env_config['df_dict'] # Daily OHCL data for each stock, indexed and aligned by day

        self._days = self._df_dict[self._tickers[0]].index
        self._trading_days = env_config['trading_days'] # Number of days the algorithm will be trading
        self._start_day_idx = env_config['start_day_offset'] # Offset of the first trading day from the first dataframe day
        
        if self._start_day_idx is not None:
            assert self._start_day_idx >= self._max_indicator_lookback, 'start_day_offset must be larger than max_indicator_lookback in order to properly initialise all indicators'
            assert self._start_day_idx + self._trading_days <= len(self._days), 'start_day_idx + trading_days must be lower than the number of days'
        else:
            self._start_day_idx
        
        assert self._trading_days + self._max_indicator_lookback <= len(self._days) ,'The sum of trading_days + max_indicator_lookback must be lower than the number of days in the dataframe'

        self._initial_balance = self._env_config['initial_balance']
        self._initial_portfolio = self._env_config['initial_portfolio'] if self._env_config['initial_portfolio'] is not None else [0] * self._n_assets

        assert len(self._initial_portfolio) == self._n_assets, 'Size of initial portfolio must equal the number of assets!'

        action_shape = (self._n_assets + 1,)
        obs_shape = (self._n_features*self._n_assets + 1,)

        self.action_space = gym.spaces.Box(np.full(action_shape, 0), np.full(action_shape, 1), shape=action_shape, dtype=np.float16) # Action space is the assets + cash for rebalancing
        self.observation_space = gym.spaces.Box(np.full(obs_shape, 0), np.inf, shape=obs_shape, dtype=np.float16) # Observation space is all the features for each asset + cash
        self.max_episode_steps = self._trading_days

    def reset(self):
        self._balance = self._initial_balance
        self._portfolio = self._initial_portfolio

        if self._start_day_idx is None:
            self._start_day_idx = np.random.randint(self._max_indicator_lookback, len(self._days) - self._trading_days) # If no start day chosen, generate a random start
        self._cur_day_idx = self._start_day_idx
        self._cur_day = self._days[self._cur_day_idx]
        self._cur_day_idx += 1 # Advance one day
        
        indicators = self._compute_indicators(self._cur_day) # Compute the indicators for the start date
        
        return np.append(indicators, self._balance) # Observation is number of indicators * number of assets + 1

    def _compute_indicators(self, day):
        features = np.empty((self._n_features*self._n_assets,))
        for (i, ticker) in enumerate(self._tickers):
            for (j, indicator) in enumerate(self._indicator_funcs):
                ticker_frame_slice = self._df_dict[ticker].loc[self._days[self._start_day_idx] - pd.Timedelta(days=1)*self._max_indicator_lookback:(day + pd.Timedelta(days=1))] # Get the relevant dataframe up until this day (inclusive)
                features[i*self._n_features + j] = indicator(ticker_frame_slice)
        return features

    def _asset_prices(self, day): # Use open prices on the current day
        prices = np.empty((self._n_assets,))
        for i, ticker in enumerate(self._tickers):
            prices[i] = self._df_dict[ticker].loc[day].open
        return prices

    def _portfolio_val(self, portfolio, balance, day):
        return np.dot(self._asset_prices(self._cur_day), portfolio) + balance
    
    def _rebalance(self, actions): # TODO: Test this more to see if it makes sense
        weightings = self._softmax(actions) # First weight is for cash

        prices = self._asset_prices(self._cur_day) # Get the open prices of assets on the current day
        portfolio_val = np.dot(prices, self._portfolio) + self._balance
        return (portfolio_val*np.divide(weightings[1:], prices), portfolio_val*weightings[0]) # Rebalanced portfolio in the form of (assets, cash)

    def _reward(self):
        # For now just compute the increase in portfolio value
        return 1 - self._portfolio_val(self._portfolio, self._balance, self._cur_day) / self._portfolio_val(self._initial_portfolio, self._initial_balance, self._days[self._start_day_idx])

    def step(self, action):
        self._cur_day = self._days[self._cur_day_idx]
        #print('Day: ' + str(self._cur_day))
        (self._portfolio, self._balance) = self._rebalance(action)

        obs = np.append(self._compute_indicators(self._cur_day), self._balance)
        rw = self._reward()
        done = (self._cur_day_idx - self._start_day_idx) >= self._trading_days
        info = {} # TODO: Add info here

        self._cur_day_idx += 1 # Advance one day
        return obs, rw, done, info
    
    def _softmax(self, x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

In [11]:
close_indicator = lambda df: df.close[-1]

In [12]:
n_assets = len(ticker_dfs.keys())
env_config = {
    'initial_balance': 1E6,
    'initial_portfolio': [0]*n_assets,
    'tickers': list(ticker_dfs.keys()), # Tickers to trade, must correspond to tickers in dataframe dict! Implicitly defines number of assets
    'indicators': [close_indicator], # Indicator functions/classes to compute features for each stock, implicitly defines number of features. TODO: Support multidimensional indicators
    'max_indicator_lookback': 0, # Number of days after which all indicators can compute proper values
    'trading_days': 100,
    'start_day_offset': None,
    'df_dict': ticker_dfs
}

In [14]:
import ray.rllib.agents.ppo as ppo
import ray.rllib.models.catalog as catalog
import ray.tune as tune
from ray.tune.logger import pretty_print

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 5
config["rollout_fragment_length"] = 100
config["train_batch_size"] = 500
#config["framework"] = "torch"
config["env_config"] = env_config
config["log_level"] = "DEBUG"
config["env"] = TradingEnv

model_config = catalog.MODEL_DEFAULTS.copy()
model_config["use_lstm"] = True
model_config["max_seq_len"] = 100

#trainer = ppo.PPOTrainer(config=config, env=TradingEnv)
tune.run(ppo.PPOTrainer, stop={"training_iteration": 100}, config=config, local_dir='ray_results')

"""
# Can optionally call trainer.restore(path) to load a checkpoint.

for i in range(1000):
    # Perform one iteration of training the policy with PPO
    result = trainer.train()
    print(pretty_print(result))
    
    checkpoint = trainer.save()
    print("checkpoint saved at", checkpoint)"""

2020-11-03 12:34:26,239	WARNING util.py:139 -- The `start_trial` operation took 3.858757972717285 seconds to complete, which may be a performance bottleneck.
2020-11-03 12:34:27,024	WARNING util.py:139 -- The `experiment_checkpoint` operation took 0.7847261428833008 seconds to complete, which may be a performance bottleneck.


Trial name,status,loc
PPO_TradingEnv_82736_00000,RUNNING,


ray((100,), dtype=int64, min=0.0, max=0.0, mean=0.0),
(pid=29361)             'value_targets': np.ndarray((100,), dtype=float32, min=0.017, max=4.384, mean=2.624),
(pid=29361)             'vf_preds': np.ndarray((100,), dtype=float32, min=-0.021, max=0.001, mean=-0.015)},
(pid=29361)   'type': 'SampleBatch'}
(pid=29361) 
(pid=29360) 2020-11-03 12:36:10,352	INFO tf_policy.py:622 -- Optimizing variable <tf.Variable 'default_policy/fc_1/kernel:0' shape=(389, 256) dtype=float32>
(pid=29360) 2020-11-03 12:36:10,353	INFO tf_policy.py:622 -- Optimizing variable <tf.Variable 'default_policy/fc_1/bias:0' shape=(256,) dtype=float32>
(pid=29360) 2020-11-03 12:36:10,353	INFO tf_policy.py:622 -- Optimizing variable <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(389, 256) dtype=float32>
(pid=29360) 2020-11-03 12:36:10,353	INFO tf_policy.py:622 -- Optimizing variable <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(256,) dtype=float32>
(pid=29360) 2020-11-03 12:36:10,353	INFO tf_policy

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_TradingEnv_82736_00000,RUNNING,192.168.0.100:29360,1,83.619,500,2.82655,6.79577,-2.0181,100


(pid=29372) 2020-11-03 12:36:41,449	INFO sample_batch_builder.py:211 -- Trajectory fragment after postprocess_trajectory():
(pid=29372) 
(pid=29372) { 'agent0': { 'data': { 'action_dist_inputs': np.ndarray((100, 778), dtype=float32, min=-0.069, max=0.069, mean=-0.0),
(pid=29372)                         'action_logp': np.ndarray((100,), dtype=float32, min=-590.546, max=-518.966, mean=-552.134),
(pid=29372)                         'action_prob': np.ndarray((100,), dtype=float32, min=0.0, max=0.0, mean=0.0),
(pid=29372)                         'actions': np.ndarray((100, 389), dtype=float32, min=-4.172, max=4.297, mean=-0.001),
(pid=29372)                         'advantages': np.ndarray((100,), dtype=float32, min=-0.847, max=1.46, mean=-0.309),
(pid=29372)                         'agent_index': np.ndarray((100,), dtype=int64, min=0.0, max=0.0, mean=0.0),
(pid=29372)                         'dones': np.ndarray((100,), dtype=bool, min=0.0, max=1.0, mean=0.01),
(pid=29372)                  

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_TradingEnv_82736_00000,RUNNING,192.168.0.100:29360,41,1687.73,20500,2.76697,7.14933,-2.19411,100


(pid=29360) 2020-11-03 13:06:39,914	INFO multi_gpu_impl.py:143 -- Training on concatenated sample batches:
(pid=29360) 
(pid=29360) { 'inputs': [ np.ndarray((500, 389), dtype=float32, min=-4.526, max=4.418, mean=0.001),
(pid=29360)               np.ndarray((500,), dtype=float32, min=-0.061, max=0.186, mean=0.029),
(pid=29360)               np.ndarray((500, 389), dtype=float32, min=1.66, max=1000000.0, mean=129.845),
(pid=29360)               np.ndarray((500, 778), dtype=float32, min=-0.264, max=0.237, mean=0.001),
(pid=29360)               np.ndarray((500,), dtype=float32, min=-603.086, max=-514.737, mean=-553.375),
(pid=29360)               np.ndarray((500, 389), dtype=float32, min=-4.526, max=4.418, mean=0.001),
(pid=29360)               np.ndarray((500,), dtype=float32, min=-9.508, max=7.902, mean=-0.0),
(pid=29360)               np.ndarray((500, 389), dtype=float32, min=1.66, max=1000000.0, mean=129.845),
(pid=29360)               np.ndarray((500, 389), dtype=float32, min=-4.526, m

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_TradingEnv_82736_00000,RUNNING,192.168.0.100:29360,42,1730.96,21000,2.77029,7.25589,-2.19411,100


(pid=29361) 2020-11-03 13:06:57,094	INFO sampler.py:800 -- Preprocessed obs: np.ndarray((389,), dtype=float64, min=8.02, max=2401.28, mean=124.043)
(pid=29361) 2020-11-03 13:06:57,095	INFO sampler.py:805 -- Filtered obs: np.ndarray((389,), dtype=float64, min=8.02, max=2401.28, mean=124.043)
(pid=29361) 2020-11-03 13:06:57,095	INFO sampler.py:1223 -- Inputs to compute_actions():
(pid=29361) 
(pid=29361) { 'default_policy': [ { 'data': { 'agent_id': 'agent0',
(pid=29361)                                   'env_id': 0,
(pid=29361)                                   'info': {},
(pid=29361)                                   'obs': np.ndarray((389,), dtype=float64, min=8.02, max=2401.28, mean=124.043),
(pid=29361)                                   'prev_action': np.ndarray((389,), dtype=float32, min=-3.056, max=2.895, mean=-0.063),
(pid=29361)                                   'prev_reward': 0.07229796272835198,
(pid=29361)                                   'rnn_state': []},
(pid=29361)       

KeyboardInterrupt: 